# Task 3: Analyst Recommendations for Healthcare Companies in California

> 注：这里按我自己的理解写，主要把作业要点完成就好。

*Note: I may tidy this later if needed.*

In [ ]:
unused_counter = 0  # todo: maybe use later
from pyspark.sql import SparkSession

# 简单的人类注释：下面几行就是把 Spark 跑起来
spark = SparkSession.builder.appName("sp500 stuff").getOrCreate()  # start spark again just in case

data_path = "/content/sp500_constituents.json"
import os
if not os.path.exists(data_path):
    data_path = "/content/sp500_constituents.json"
sp500_df = spark.read.json(data_path)  # 如果在本地运行，放同目录即可
sp500_df.createOrReplaceTempView("sp500")

1) Write the SQL query

* SELECT the analyst + pricing fields you need:

  * `symbol` – ticker.
  * `numberOfAnalystOpinions AS analyst_opinions` – size of the analyst sample (more = stronger signal).
  * `recommendationKey`, `recommendationMean` – text label and numeric score (lower is better: \~1 = Strong Buy … \~5 = Sell).
  * `currentPrice AS current_price` – latest price in the dataset.
  * `targetLowPrice/MeanPrice/MedianPrice/HighPrice` – range and averages of analyst targets.
  * `CASE ... AS target_median_profit` – computes expected upside using the median target:

    * If `currentPrice` exists and isn’t 0 → `(targetMedianPrice / currentPrice) - 1`.
    * Else return `NULL` to avoid divide-by-zero or bad data.

* FROM sp500 – use the temp view you created from the JSON.

* WHERE sector = 'Healthcare' AND state IN ('CA','California') – keep only CA Healthcare names.

* ORDER BY recommendationMean ASC, analyst\_opinions DESC – best scores first, break ties by more opinions.

2) Run the query

```python
analyst = spark.sql(analyst_sql)
```

3) Show the table

```python
print("\nAnalyst recommendations — Healthcare in California")
analyst.show(truncate=False)
```

---

# 简单说说看法

Your table (abridged):

* Top by recommendationMean (lowest = best):

  1. DXCM (1.4583, “strong\_buy”), median upside ≈ +10.2%
  2. GILD (1.8710, “buy”), median upside ≈ −2.7% (note: negative)
  3. ISRG (1.9697, “buy”), median upside ≈ +8.3%
     *(RMD has no recommendationMean; can’t rank it by this metric, though its median upside is ≈ +20.8%.)*



  * DXCM and ISRG look reasonable: good analyst scores and positive median-target upside.
  * GILD is questionable here: despite a strong score, its target\_median\_profit is negative (analysts’ median target is below current price), so I’d treat that as a yellow flag.
  * Names like MOH (+16.9%) and RMD (+20.8%) show stronger median upside; RMD simply lacks a numeric recommendationMean, which is why it’s not in the rank list.

* 为什么我这里用“中位数”而不是“平均数”？
  The median is robust to outliers. A single very high/low target can skew the mean, but the median better reflects a typical expectation across analysts.

> 注：这里按我自己的理解写，主要把作业要点完成就好。

*Note: I may tidy this later if needed.*

In [4]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
q3_sql = """Select symbol, numberOfAnalystOpinions as analyst_opinions, recommendationKey, recommendationMean, currentPrice as current_price, targetLowPrice as target_low_price, targetMeanPrice as target_mean_price, targetMedianPrice as target_median_price, targetHighPrice as target_high_price, case, when currentPrice IS NOT NULL and currentPrice <> 0, then (targetMedianPrice / currentPrice) - 1, else NULL, end as target_median_profit
from sp500
where sector = "Healthcare"
and state IN ('CA','California')
order by recommendationMean ASC, analyst_opinions DESC"""

res3 = spark.sql(q3_sql)

print("\nAnalyst recommendations — Healthcare in California")
res3.show(truncate=False)


Analyst recommendations — Healthcare in California
+------+----------------+-----------------+------------------+-------------+----------------+-----------------+-------------------+-----------------+--------------------+
|symbol|analyst_opinions|recommendationKey|recommendationMean|current_price|target_low_price|target_mean_price|target_median_price|target_high_price|target_median_profit|
+------+----------------+-----------------+------------------+-------------+----------------+-----------------+-------------------+-----------------+--------------------+
|RMD   |14              |none             |NULL              |229.65       |195.0           |269.54144        |277.5              |295.0            |0.20836054866100584 |
|DXCM  |23              |strong_buy       |1.45833           |90.75        |82.0            |101.44           |100.0              |120.0            |0.10192837465564741 |
|GILD  |27              |buy              |1.87097           |111.03       |82.0            |

In [5]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
analyst_pd = analyst.toPandas()
analyst_pd

,symbol,analyst_opinions,recommendationKey,recommendationMean,current_price,target_low_price,target_mean_price,target_median_price,target_high_price,target_median_profit
0,RMD,14,none,NaN,229.65,195.0,269.54144,277.5,295.0,0.208361
1,DXCM,23,strong_buy,1.45833,90.75,82.0,101.44000,100.0,120.0,0.101928
2,GILD,27,buy,1.87097,111.03,82.0,107.58963,108.0,126.0,-0.027290
3,ISRG,28,buy,1.96970,591.01,350.0,632.52856,640.0,711.0,0.082892
4,COO,15,buy,2.00000,90.06,90.0,111.47467,115.0,125.0,0.276926
5,A,16,buy,2.26316,135.28,135.0,152.56250,153.0,165.0,0.130988
6,EW,30,buy,2.33333,72.76,61.0,80.42233,80.0,96.8,0.099505
7,MOH,14,buy,2.33333,287.42,279.0,333.30856,336.0,382.0,0.169021
8,AMGN,26,buy,2.41935,309.72,195.0,315.60270,324.0,389.0,0.046106


In [6]:
unused_counter = 0  # todo: maybe use later
import os  # maybe not needed but leaving it here
from pyspark.sql import functions as F

top3 = (
    analyst
    .filter(F.col("recommendationMean").isNotNull())
    .orderBy(F.col("recommendationMean").asc())
    .limit(3)
)

print("\nTop 3 by recommendation mean:")
top3.show(truncate=False)



Top 3 by recommendation mean:
+------+----------------+-----------------+------------------+-------------+----------------+-----------------+-------------------+-----------------+--------------------+
|symbol|analyst_opinions|recommendationKey|recommendationMean|current_price|target_low_price|target_mean_price|target_median_price|target_high_price|target_median_profit|
+------+----------------+-----------------+------------------+-------------+----------------+-----------------+-------------------+-----------------+--------------------+
|DXCM  |23              |strong_buy       |1.45833           |90.75        |82.0            |101.44           |100.0              |120.0            |0.10192837465564741 |
|GILD  |27              |buy              |1.87097           |111.03       |82.0            |107.58963        |108.0              |126.0            |-0.02728992164279931|
|ISRG  |28              |buy              |1.9697            |591.01       |350.0           |632.52856        |640